In [1]:
# Using the REST API we define a function to get a custom role ID by name
import requests
import json
import pandas

# Note: this does NOT depend on the arcgis python API, but you can certainly integrate it with code that does. 

# We need to get a token first before we can make the REST API request. 
def generateToken(username, password, portalUrl):
    ''' Retrieves a token to be used with API requests.
        Adapted from the urllib example at http://enterprise.arcgis.com/en/portal/latest/administer/windows/scripting-with-the-arcgis-rest-api.htm
        to use the requests module.
    '''
    parameters = {  'client'    : 'referer',
                    'username'  : username,
                    'password'  : password,
                    'referer'   : portalUrl,
                    'expiration': 60,
                    'f'         : 'json'}
    
    response = requests.post(portalUrl+"/sharing/rest/generateToken",data=parameters)
    try:
        jsonResponse = json.loads(response.text)
        if 'token' in jsonResponse:
            return jsonResponse['token']
        elif 'error' in jsonResponse:
            print (jsonResponse['error']['message'])
            for detail in jsonResponse['error']['details']:
                print (detail)
    except ValueError as e:
        print ('An unspecified error occurred.')
        print (e)
        
def get_roles(token, portalUrl):
    parameters={'token':token,'f':'json','num':100}  #NOTE: if you have more than 100 custom roles you'll need to paginate results
    response = requests.get(portalUrl+"/sharing/rest/portals/self/roles", params=parameters)
    if(response.ok):
        jsonResponse = json.loads(response.content)
        roleMappings = jsonResponse['roles']
        return roleMappings
    
# Step 1: Load global config with auth credentials
# (you could also get these interactively):
with open('vtActiveConfig.json') as configFile:
    myConfig = json.load(configFile)
    
# Step 2: generate the auth token
token = generateToken(myConfig['agolOrg']['username'], myConfig['agolOrg']['password'], myConfig['agolOrg']['url'])

# Step 3: Retrieve all Custom roles
customRoles = get_roles(token,myConfig['agolOrg']['url'])
df = pandas.DataFrame((customRoles))

# Step 4ascii: lookup the role you want
def getRoleId(roleName,sourceRoleDf):
    return sourceRoleDf.loc[sourceRoleDf['name'] == roleName, 'id'].item()



# Call the function
# Substitute the name of any role you want to lookup for "New_User"


print(getRoleId("New_User",df))

pH1lPndPVtVbrE6l
